<a href="https://colab.research.google.com/github/silverstar0727/n_body_problem/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [ ]:
# data load
batch_size = 256

mnist_train = dset.MNIST("./", train = True, transform = transforms.ToTensor(),
                         target_transform = None, download = True) # "./"는 현재 코드가 있는 위치를 경로로 사용하겠다는 의미
mnist_test = dset.MNIST("./", train = False, transform = transforms.ToTensor(),
                        target_transform = None, download = True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size,
                                          shuffle = True, num_workers = 2, 
                                          drop_last = True) 
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = batch_size, shuffle = True,
                                          num_workers = 2, drop_last = True)

In [ ]:
# modeling
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(1, 16, 5),
        nn.ReLU(),
        nn.Conv2d(16, 32, 5),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Conv2d(32, 64, 5),
        nn.ReLU(),
        nn.MaxPool2d(2, 2)
    )

    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(batch_size, -1)
    out = self.fc_layer(out)
    return out

In [ ]:
# loss function, optimizer, put model on device
learning_rate = 0.0002

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  # cuda가 있으면 cuda로 없으면 cpu로
model = CNN().to(device) # CNN모델을 device에 놓기
loss_func = nn.CrossEntropyLoss() # 크로스 엔트로피를 loss function으로 사용
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # adam optimizer 사용

In [ ]:
# 반복
loss_arr = [] # loss값을 기록
num_epoch = 10 # 10 epochs
for i in range(num_epoch):
  for j, [image, label] in enumerate(train_loader):
    x = image.to(device) # x data를 device에 놓기
    y_ = label.to(device) # y data를 device에 놓기

    optimizer.zero_grad() # 극솟값찾기
    output = model.forward(x) # 순전파의 과정
    loss = loss_func(output, y_) # 오차를 정의
    loss.backward() # 오차 역전파
    optimizer.step() # 매개변수 변경

    if j % 1000 == 0: # 1000번반복 할 때 마다
      print(loss) # loss를 출력하고
      loss_arr.append(loss.cpu().detach().numpy()) # 저장

tensor(2.3142, grad_fn=<NllLossBackward>)
tensor(0.2893, grad_fn=<NllLossBackward>)
tensor(0.2196, grad_fn=<NllLossBackward>)
tensor(0.1234, grad_fn=<NllLossBackward>)
tensor(0.0742, grad_fn=<NllLossBackward>)
tensor(0.0880, grad_fn=<NllLossBackward>)
tensor(0.0336, grad_fn=<NllLossBackward>)
tensor(0.0575, grad_fn=<NllLossBackward>)
tensor(0.0364, grad_fn=<NllLossBackward>)
tensor(0.0386, grad_fn=<NllLossBackward>)


In [ ]:
correct = 0
total = 0

with torch.no_grad(): # grad의 업데이트 없이 진행, 모델에 대한 검증시에 사용
  for image, label in test_loader:
    x = image.to(device) # image를 device에 올려 x에 저장
    y_ = label.to(device) # True값을 device에 올려 y_에 저장

    output = model.forward(x) # x data를 모델에 순전파 하면서 결과를 output에 저장
    print(torch.max(output, 1)) 
    _, output_index = torch.max(output, 1)

    total += label.size(0)
    correct += (output_index == y_).sum().float()

  print('Accuracy of Test Data: {}', format(100 * current / total))

torch.return_types.max(
values=tensor([17.4330, 17.2216, 12.1282, 11.4282, 13.8053, 10.4714, 11.4014, 17.2459,
        11.5809, 13.7179, 20.8320, 16.0422, 19.4898, 16.9256, 16.3074,  8.4244,
         9.5227,  8.8488, 18.6975, 17.1869, 11.3859, 15.7243, 15.5514, 19.4316,
        13.5485, 10.1582, 13.9538, 14.5458, 10.6900,  8.8427, 19.6319, 14.3635,
        20.2304, 10.8781,  9.4839, 15.6141, 16.8427, 13.2074, 18.9946, 14.2545,
        14.9548, 11.6853,  9.9136, 12.2408, 17.8228,  7.2421, 13.1572, 14.1773,
         9.6458,  8.9083, 15.4470, 14.0930, 10.3840,  9.6864, 15.2339,  8.9203,
        18.5195, 11.4848, 15.6373, 16.6471, 10.1848, 17.1691, 12.1268, 10.2381,
        17.2921,  7.9030,  6.8169, 20.6548, 15.2030, 11.7900, 15.7002, 16.1753,
        20.2639, 16.3519, 13.7086,  7.6285, 17.2232, 10.9299,  8.8529, 12.2089,
         9.2295, 16.4686, 18.5130, 12.0016, 14.9637, 13.2804, 21.2661, 13.9239,
        17.1031, 14.6720, 12.8233, 11.8484, 12.0527, 10.5810, 19.0770, 12.0715,
         

## VGGNet

In [ ]:
def conv_2_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size = 3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(2, 2)
  )
  
  return model

def conv_3_block(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, kernel_size = 3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
      nn.ReLU(),
      nn.Conv2d(out_dim, out_dim, kernel_size = 3, padding = 1),
      nn.ReLU(),
      nn.MaxPool2d(2, 2)
  )

  return model

class VGG(nn.Module):
  def __init__(self, base_dim, num_classes = 2):
    super(VGG, self).__init__()
    self.feature = nn.Sequential(
        conv_2_block(3, base_dim),
        conv_2_block(base_dim, 2*base_dim),
        conv_3_block(2*base_dim, 4*base_dim),
        conv_3_block(4*base_dim, 8*base_dim),
        conv_3_block(8*base_dim, 8*base_dim),
    )

    self.fc_layer = nn.Sequential(
        nn.Linear(8*base_dim*7*7, 100),
        nn.ReLU(),
        nn.Linear(100, 20),
        nn.ReLU(),
        nn.Linear(20, num_classes),
    )

    def forward(self, x):
      x = self.feature(x)
      x = x.view(x.size(0), -1)
      x = self.fc_layer(x)
      return x

## GoogLeNet

In [ ]:
def conv_1(in_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, out_dim, 1, 1),
      nn.ReLU(),
  )

  return model

def conv_1_3(in_dim, mid_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, mid_dim, kernel_size = 1, stride = 1),
      nn.ReLU(),
      nn.Conv2d(in_dim, out_dim, kernel_size = 3, stride = 1, padding = 1),
      nn.ReLU()
  )

  return model

def conv_1_5(in_dim, mid_dim, out_dim):
  model = nn.Sequential(
      nn.Conv2d(in_dim, mid_dim, 1, 1),
      nn.ReLU(),
      nn.Conv2d(mid_dim, out_dim, 5, 1, 1),
      nn.ReLU()
  )

  return model

def max_3_1(in_dim, out_dim):
  model = nn.Sequential(
      nn.MaxPoo2d(3, 1, 1),
      nn.Conv2d(in_dim, out_dim, 1, 1),
      nn.ReLU(),
  )

  return model

In [ ]:
class inception_module(nn.Module):
  def __init__(self, in_dim, out_dim, mid_dim_3, out_dim_3, mid_dim_5, out_dim_5, pool):
    super(inception_module, self).__init__()

    self.conv_1 = conv_1(in_dim, out_dim_1)
    self.conv_1_3 = conv_1_3(in_dim, mid_dim_3, out_dim_3)
    self.conv_1_5 = conv_1_5(in_dim, mid_dim_5, out_dim_5)
    self.max_3_1 = max_3_1(in_dim, pool)

  def forward(self, x):
    out_1 = self.conv_1(x)
    out_2 = self.conv_1_3(x)
    out_3 = self.conv_1_5(x)
    out_4 = self.max_3_1(x)
    output = torch.cat([out_1, out_2, out_3, out_4])
    return output

In [ ]:
class GoogLeNet(nn.Module):
  def __init__(self, base_dim, num_classes = 2):
    super(GoogLeNet, self).__init__()
    self.layer_1 = nn.Sequential(
        nn.Conv2d(3, base_dim, 7, 2, 3)
        nn.MaxPool
    )